In [ ]:
import pandas as pd
from sklearn.decomposition import PCA

# PUT THIS IMPORT BACK IN LATER!! There is something messed up my pip data for matplotlib for now
# import matplotlib.pyplot as plt

from sklearn.preprocessing import MinMaxScaler

# for displaying text with formatting later
from IPython.display import display, HTML


In [ ]:
# import the utility functions for the World Bank and OECD.
# It requires changing the path to import properly in Python
import sys
sys.path.append('../utility_functions')

from world_bank_oecd_utility_functions import (get_indicator_name_from_code, get_indicator_definition_or_additional_info_from_code)

## Change the variable OECD below to True if you want to use the OECD dataset or False if you want to use the World Bank dataset

In [ ]:
# Select data
datasets = ["../OECD/Cleaned/HEALTH_MERGED_Threshold_80_n3.csv", "../WorldBankDatasets/Cleaned/AllMerged_Threshold_85_n3_MMRTNE.csv"]

# OECD = False  # False for WB data
OECD = True

mapping_df = ''

mapping_oecd_to_names_and_additional_info_df = pd.read_csv('../OECD/Cleaned/OECD_Indicator_Definition_Info.csv',
                                                             # delimiter=',')
                                                             delimiter=',')
mapping_world_bank_to_names_and_definitions_df = pd.read_csv('../WorldBankDatasets/Cleaned/World_Bank_Indicator_Definition_Info.csv',
                                                             delimiter='\t')

# remove an annoying column if it exists in the mapping for World Bank
if "Unnamed: 3" in mapping_world_bank_to_names_and_definitions_df.columns:
    mapping_world_bank_to_names_and_definitions_df = mapping_world_bank_to_names_and_definitions_df.drop(columns=["Unnamed: 3"])

if OECD:
    mapping_df = mapping_oecd_to_names_and_additional_info_df
    df = pd.read_csv(datasets[0])
    mm_ind = "MATIMATM"
else:
    mapping_df = mapping_world_bank_to_names_and_definitions_df
    df = pd.read_csv(datasets[1])
    mm_ind = "SH.STA.MMRT.NE"

In [ ]:
# df

In [ ]:
# uncomment to see the oecd mapping csv that maps each indicator code to the name and additional information
# mapping_oecd_to_names_and_additional_info_df

In [ ]:
# uncomment to see the world bank mapping csv that maps each indicator code to the name and definition
# mapping_world_bank_to_names_and_definitions_df

In [ ]:
# mapping_df.columns

In [ ]:
# testing cell to test the oecd and world bank mapping functions. Change OECD binary value to test the either oecd or world bank
# sets

# OECD = False
# # OECD = True

# if OECD:
#     mapping_df = mapping_oecd_to_names_and_additional_info_df
#     df = pd.read_csv(datasets[0])
#     mm_ind = "MATIMATM"
# else:
#     mapping_df = mapping_world_bank_to_names_and_definitions_df
#     df = pd.read_csv(datasets[1])
#     mm_ind = "SH.STA.MMRT.NE"

# # Test switching
# if (OECD):
#     code = 'ECONAICO'

# else:
#     code = 'SH.ANM.NPRG.ZS'
# test1  = get_indicator_name_from_code(code, mapping_df)
# print(test1)
# test2 = get_indicator_definition_or_additional_info_from_code(code, mapping_df)
# print(test2)

## Data Preparation

In [ ]:
# Reduce to only gender indicators

#if not OECD:
#    gender_ind = pd.read_csv('../WorldBankDatasets/Gender_WorldBankData.csv').columns
#    df = df.drop(columns=[col for col in df if col not in gender_ind])

#df.head()

In [ ]:
# Select year
year = 2015
df = df[df["Year"] == year]
df.drop("Year", axis=1, inplace=True)

# Scale predictor, which is currently unscaled
scaler = MinMaxScaler()
df[mm_ind] = scaler.fit_transform(df[mm_ind].to_numpy().reshape(-1, 1))

# Drop country
X = df.drop(columns=['Country'])

## Dimensionality Reduction

# PCA

In [ ]:
pca = PCA(n_components=2, random_state=42)
pca.fit(X)

print("Number of PCs: {}".format(len(pca.explained_variance_ratio_)))
print("Explained variation per PC: {}".format(pca.explained_variance_ratio_))
print("Sum of explained variation: {}".format(pca.explained_variance_ratio_.sum()))

# Now to show you detailed information of the features that contribute to the PC analysis

In [ ]:
# note this only works one loading set at a time. For example if I had
# pca_loadings = pca.components_
# the code would only work for 1 set, for example pca_loadings[0]
def sort_loading_set_for_principal_component(loadings, X=X):
    feature_names = X.columns
    
    # make a series of the current pca loadings. Index is the column name (feature) and value is the loading value
    feature_names_series = pd.Series(loadings, index=feature_names)

    # take the absolute value of the series
    feature_names_series_abs = feature_names_series.abs()

    # finally sort descending. The higher the number, the higher the correlatino and explanatory value of the 
    # feature for the principal component
    pca_component_explanatory_features_sorted = feature_names_series_abs.sort_values(ascending=False)
    
    return pca_component_explanatory_features_sorted

In [ ]:
# uncomment below to test the function sort_loading_set_for_principal_component

# percentage_of_variance_to_explain = 0.8
# pca = PCA(n_components=percentage_of_variance_to_explain, random_state=42)
# pca.fit(X)
# pca_loadings = pca.components_

# sort_loading_set_for_principal_component(pca_loadings[0])

In [ ]:
# now works for World Bank or OECD
def get_most_important_features_from_pca_loading_set (pca_loading_set, mapping_df,
                                                   n_most_important_features_from_loading = 5, 
                                                   includeName = True, includeDefinitionOrAdditionalInfo = True):
    pca_loadings_explanations = {}

    # get just the top n_most_important_features_from_loading features
    cur_loadings_sorted_with_n_most_important_features = sort_loading_set_for_principal_component(pca_loading_set).iloc[
        0:n_most_important_features_from_loading]

    feature_codes = []
    feature_values = []
    feature_names = []
    feature_definitions_or_additional_info = []

    for feature_code_index, feature_value in zip (cur_loadings_sorted_with_n_most_important_features.index,
                                                  cur_loadings_sorted_with_n_most_important_features.values):
        feature_codes.append(feature_code_index)
        feature_values.append(feature_value)

        if (includeName):
            cur_name = get_indicator_name_from_code(feature_code_index, mapping_df)
            # print('cur_name is', cur_name)
            feature_names.append(cur_name)
        
        if (includeDefinitionOrAdditionalInfo):
            cur_definition_or_additional_info = get_indicator_definition_or_additional_info_from_code (feature_code_index,
                                                                                                mapping_df)
            feature_definitions_or_additional_info.append(cur_definition_or_additional_info)

    pca_loadings_explanations["most_important_feature_values"] = feature_values
    pca_loadings_explanations["most_important_codes"] = feature_codes
    pca_loadings_explanations["most_important_feature_names"] = feature_names
    pca_loadings_explanations["most_important_definitions_or_additional_info"] = feature_definitions_or_additional_info

    return pca_loadings_explanations

In [ ]:
# uncomment below to test the function get_most_important_features_from_pca_loading_set

# percentage_of_variance_to_explain = 0.8
# pca = PCA(n_components=percentage_of_variance_to_explain, random_state=42)
# pca.fit(X)
# pca_loadings = pca.components_

# get_most_important_features_from_pca_loading_set(pca_loadings[0], mapping_df)

In [ ]:
# get_most_important_features_from_pca_loading_set(pca_loadings[1], mapping_df)

In [ ]:
# this function will show the top most important components explaining the variance for each of the pcs 
# give a given percentage of variance to explain
# now supports either world bank or OECD
def pca_with_detailed_variance_and_components_info (percentage_of_variance_to_explain, mapping_df, X=X,
                                                              n_most_important_features_from_loading=5,
                                                              includeValues = False, includeDefinitionOrAdditionalInfo=False):
    
    pca = PCA(n_components=percentage_of_variance_to_explain, random_state=42)
    pca.fit(X)
    pca_loadings = pca.components_

    loadings_information = []
    for loading in pca_loadings:
        cur_loading_features_info = get_most_important_features_from_pca_loading_set (loading, mapping_df,
                                                                                  n_most_important_features_from_loading)
        loadings_information.append(cur_loading_features_info)
    
    num_principal_components = len(pca.explained_variance_ratio_)

    print(f'in order to explain {percentage_of_variance_to_explain * 100} percentage of the variance with pca it requires')
    print(f'{num_principal_components} principal components')
    print(f'the most important feature names, for the top {n_most_important_features_from_loading} from each PC, are\n')

    # print the most important value names
    for i in range(0, len(loadings_information)):
        # print('\n')
        print('---------')
        print(f'For Principal Component {i+1}, the most important features names are:\n')
        # print('\n')
        cur_loadings_info = loadings_information[i]

        cur_loadings_names = cur_loadings_info['most_important_feature_names']
        cur_loadings_values = cur_loadings_info['most_important_feature_values']
        cur_loadings_codes = cur_loadings_info['most_important_codes']
        cur_loadings_definitions_or_additional_info = cur_loadings_info['most_important_definitions_or_additional_info']
        # most_important_definitions_or_additional_info

        for code, value, name, definition_or_additional_info in zip(cur_loadings_codes, cur_loadings_values, cur_loadings_names,
                                                 cur_loadings_definitions_or_additional_info):
            print(name)
            
            # uncomment for testing to debug
            # print(code)

            if (includeValues):
                print(f"with value {value}")

            if (includeDefinitionOrAdditionalInfo):
                print(definition_or_additional_info)
        print('\n')

## Modify variance_percentage_to_explain below for detailed information based on what percentage of the variance in principal components you want to explain maternal mortality. (see section below for just the top feature per PC)

In [ ]:
variance_percentage_to_explain = 0.8
# variance_percentage_to_explain = 0.9
num_top_features_per_pc = 5
pca_with_detailed_variance_and_components_info (variance_percentage_to_explain, mapping_df,
                                                includeValues=True,
                                                n_most_important_features_from_loading = num_top_features_per_pc)

In [ ]:
# returns a list of the top feature name for each principal component
def get_top_feature_for_each_pc (percentage_of_variance_to_explain, X=X, n_most_important_features_from_loading=5):
    pca = PCA(n_components=percentage_of_variance_to_explain, random_state=42)
    pca.fit(X)
    pca_loadings = pca.components_

    loadings_information = []
    for loading in pca_loadings:
        cur_loading_features_info = get_most_important_features_from_pca_loading_set(loading, mapping_df,
                                                                                  n_most_important_features_from_loading)
        loadings_information.append(cur_loading_features_info)
    
    top_feature_name_per_pc = []
    for i in range(0, len(loadings_information)):
        cur_loadings_info = loadings_information[i]

        cur_features_names = cur_loadings_info['most_important_feature_names']
        cur_top_feature = cur_features_names[0]
        top_feature_name_per_pc.append(cur_top_feature)
        # cur_loadings_values = cur_loadings_info['most_important_features_values']
        # cur_loadings_codes = cur_loadings_info['most_important_features_world_bank_codes']
        # cur_loadings_definitions = cur_loadings_info['most_important_features_world_bank_definitions']
    
    return top_feature_name_per_pc

## Modify variance_percentage_to_explain below based on what percentage of the variance in principal components. You may find the results interesting!

In [ ]:
# below starts to use html formatting

In [ ]:
# variance_percentage_to_explain = 0.2
# variance_percentage_to_explain = 0.3
# variance_percentage_to_explain = 0.5
# variance_percentage_to_explain = 0.8
# variance_percentage_to_explain = 0.85
# variance_percentage_to_explain = 0.9
variance_percentage_to_explain = 0.95

top_feature_names_for_pc_loadings = get_top_feature_for_each_pc(variance_percentage_to_explain)

# print(f"""Assuming you want to explain {variance_percentage_to_explain * 100} percent of the PC variation, 
# for maternal morality, the top feature names that explain this per principal component are:\n""")

if OECD:
    dataset_using = 'OECD'
else:
    dataset_using = 'World Bank Dataset'

display( HTML(f"""<h2>Assuming you want to explain {variance_percentage_to_explain * 100} percent of the PC variation in the {dataset_using}, 
              the top feature names that explain this per principal component are:</h2>\n"""))

for i in range(0, len(top_feature_names_for_pc_loadings)):
    #print(f"for PC {i+1} the top explaining feature is: {top_feature_names_for_pc_loadings[i]}")
    # print(f"PC {i+1}: {top_feature_names_for_pc_loadings[i]}")
    display( HTML( f"<strong>PC {i+1}:</strong> {top_feature_names_for_pc_loadings[i]}" ) )
    # print("\n")